In [22]:
import os
os.getenv("LD_LIBRARY_PATH")

'/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/cv2/../../lib64::/home/annazan/miniconda3/envs/fair38/lib/'

In [23]:
!echo $LD_LIBRARY_PATH
!source ~/.bashrc
!echo $LD_LIBRARY_PATH


/bin/bash: /home/annazan/miniconda3/envs/fair38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/cv2/../../lib64::/home/annazan/miniconda3/envs/fair38/lib/
/bin/bash: /home/annazan/miniconda3/envs/fair38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/annazan/miniconda3/envs/fair38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/cv2/../../lib64::/home/annazan/miniconda3/envs/fair38/lib/


In [24]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [36]:
import os
import sys
print(os.getcwd())
os.environ.update(os.environ)
        # Add a new environment variable to the operating system
os.environ["RAMP_HOME"] = os.getcwd()
# Print the environment variables to verify that the new variable was added
print(os.environ["RAMP_HOME"])
sys.path.append('../')
sys.path.append('../ramp-code/')
sys.path.append('ramp-code')


/home/annazan/fAIr-utilities
/home/annazan/fAIr-utilities


In [26]:
import cv2

In [27]:
os.environ["SM_FRAMEWORK"] = "tf.keras"
import ramp.utils
import hot_fair_utilities
# base_path = f"{os.getcwd()}/ramp-data/sample_2"
# base_path = "/home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar"
base_path = f"{os.getcwd()}/ramp-data/test_data/1_Zanzibar"

In [28]:
from hot_fair_utilities import preprocess
model_input_image_path = f"{base_path}/input"
preprocess_output=f"{base_path}/preprocessed"
preprocess(
            input_path = model_input_image_path,
            output_path = preprocess_output,
            rasterize=True,
            rasterize_options=["binary"],
            georeference_images=True,
        )

Georeferencing for input: 100%|██████████| 10/10 [00:00<00:00, 83.32it/s]
/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
Clipping labels for input:   0%|          | 0/10 [00:00<?, ?it/s]/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/annazan/miniconda3/envs/fair38/lib/python3.8/site-packages/geopandas/io/

In [29]:
from hot_fair_utilities import train

In [37]:
train_output = f"{base_path}/train"
final_accuracy, final_model_path = train(
    input_path=preprocess_output,
    output_path=train_output,
    epoch_size=4,
    batch_size=2,
    model="ramp",
    model_home=os.environ["RAMP_HOME"]
)

variables are /home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/preprocessed
 and /home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train
Starting to prepare data for training
ramp home is /home/annazan/fAIr-utilities
python home is None
variables are: src /home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/preprocessed
 and dst:/home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train


Writing /home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train/fair_split_train.csv
Writing /home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train/fair_split_val.csv
Traceback (most recent call last):
  File "/home/annazan/fAIr-utilities/ramp-code/scripts/move_chips_from_csv.py", line 15, in <module>
    from ramp.utils.file_utils import move_training_data_files
ModuleNotFoundError: No module named 'ramp'


CalledProcessError: Command '['python', '/home/annazan/fAIr-utilities/ramp-code/scripts/move_chips_from_csv.py', '-sd', '/home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train/chips', '-td', '/home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train/val-chips', '-csv', '/home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar/train/fair_split_val.csv', '-mv']' returned non-zero exit status 1.

# here a tf file is created (weights + structure)

In [ ]:
print(final_accuracy,final_model_path)

In [ ]:
from hot_fair_utilities import predict
prediction_output = f"{base_path}/prediction/output"
predict(
    checkpoint_path=final_model_path,
    input_path=f"{base_path}/prediction/input",
    prediction_path=prediction_output,
)

In [ ]:
from hot_fair_utilities import polygonize
geojson_output= f"{prediction_output}/prediction.geojson"
polygonize(
    input_path=prediction_output, 
    output_path=geojson_output,
    remove_inputs = True,
)